In [1]:
import text_normalizer as tn
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import dill
import gc

SEED = 42
np.random.seed(SEED)
tf.set_random_seed(SEED)

/home/redanalyze/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('./data/gokube_phase1_jun19/GH_complete_labeled_issues_prs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152151 entries, 0 to 152150
Data columns (total 2 columns):
description    152151 non-null object
label          152151 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [3]:
docs = df['description'].values
labels = df['label'].values
labels = np.array([0 if item == 0 else 1 for item in labels])

In [4]:
%%time

norm_docs = tn.pre_process_documents_parallel(documents=docs)
print(len(norm_docs))

preprocessing: starting
ThreadPoolExecutor-0_0: working on doc num: 0
ThreadPoolExecutor-0_2: working on doc num: 5000
ThreadPoolExecutor-0_13: working on doc num: 10000
ThreadPoolExecutor-0_14: working on doc num: 15000
ThreadPoolExecutor-0_11: working on doc num: 20000
ThreadPoolExecutor-0_0: working on doc num: 25000
ThreadPoolExecutor-0_3: working on doc num: 30000
ThreadPoolExecutor-0_3: working on doc num: 35000
ThreadPoolExecutor-0_20: working on doc num: 40000
ThreadPoolExecutor-0_28: working on doc num: 45000
ThreadPoolExecutor-0_29: working on doc num: 50000
ThreadPoolExecutor-0_23: working on doc num: 55000
ThreadPoolExecutor-0_24: working on doc num: 60000
ThreadPoolExecutor-0_5: working on doc num: 65000
ThreadPoolExecutor-0_27: working on doc num: 70000
ThreadPoolExecutor-0_26: working on doc num: 75000
ThreadPoolExecutor-0_28: working on doc num: 80000
ThreadPoolExecutor-0_5: working on doc num: 85000
ThreadPoolExecutor-0_7: working on doc num: 90000
ThreadPoolExecutor-0

In [5]:
with open('./data/gokube_phase1_jun19/gh_preprocessed_descriptions.pkl', 'wb') as f:
    dill.dump(norm_docs, f)
    
with open('./data/gokube_phase1_jun19/gh_labels.pkl', 'wb') as f:
    dill.dump(labels, f)

In [6]:
with open('./data/gokube_phase1_jun19/gh_preprocessed_descriptions.pkl', 'rb') as f:
    norm_docs = dill.load(f)
    
with open('./data/gokube_phase1_jun19/gh_labels.pkl', 'rb') as f:
    labels = dill.load(f)

# Train on 75:25 Split

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(norm_docs, labels, 
                                                    test_size=0.25, random_state=SEED)
len(X_train), len(X_test)

(114113, 38038)

In [9]:
## some config values 
EMBED_SIZE = 300 # how big is each word vector
MAX_FEATURES = 800000 # how many unique words to use (i.e num rows in embedding vector)
MAX_LEN = 1000 # max number of words in a doc to use

In [10]:
import os.path

CVE_WORD2IDX_MAP_FILE = 'models/v3-jun19/embeddings/security_tokenizer_word2idx.pkl'

if not os.path.isfile(CVE_WORD2IDX_MAP_FILE):
    tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>', num_words=MAX_FEATURES+1)
    tokenizer.fit_on_texts(list(X_train))
    tokenizer.word_index['<PAD>'] = 0
    with open(CVE_WORD2IDX_MAP_FILE, 'wb') as f:
        dill.dump(tokenizer.word_index, f)
else:
    tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>', num_words=MAX_FEATURES+1)
    with open(CVE_WORD2IDX_MAP_FILE, 'rb') as f:
        word2idx = dill.load(f)
    tokenizer.word_index = word2idx

In [11]:
MAX_FEATURES = len(tokenizer.word_index)
MAX_FEATURES

542585

In [12]:
## Tokenize the sentences
train_X = tokenizer.texts_to_sequences(X_train)
test_X = tokenizer.texts_to_sequences(X_test)

In [13]:
## Pad the sentences 
train_X = keras.preprocessing.sequence.pad_sequences(train_X, maxlen=MAX_LEN)
test_X = keras.preprocessing.sequence.pad_sequences(test_X, maxlen=MAX_LEN)

In [14]:
train_X_lengths = np.array([len(np.nonzero(item)[0]) for item in train_X])
train_X_idx = np.argwhere(train_X_lengths >= 5).ravel()
train_X = train_X[train_X_idx]
train_y = y_train[train_X_idx]

test_X_lengths = np.array([len(np.nonzero(item)[0]) for item in test_X])
test_X_idx = np.argwhere(test_X_lengths >= 5).ravel()
test_X = test_X[test_X_idx]
test_y = y_test[test_X_idx]
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((112487, 1000), (112487,), (37468, 1000), (37468,))

In [15]:
word2idx = tokenizer.word_index

In [16]:
def load_pretrained_embeddings(word_to_index, max_features, embedding_size, embedding_file_path):    
    
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    embeddings_index = dict(get_coefs(*row.split(" ")) 
                                for row in open(embedding_file_path, encoding="utf8", errors='ignore') 
                                    if len(row)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_to_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
    
    for word, idx in word_to_index.items():
        if idx >= max_features: 
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[idx] = embedding_vector

    return embedding_matrix

In [98]:
FASTTEXT_INIT_EMBEDDINGS_FILE = './models/v3-jun19/embeddings/security_model_fasttext_init_embeddings.pkl'

if not os.path.isfile(FASTTEXT_INIT_EMBEDDINGS_FILE):
    FASTTEXT_EMBEDDINGS_PATH = './embeddings/fasttext/crawl-300d-2M.vec'
    ft_embeddings = load_pretrained_embeddings(word_to_index=word2idx, max_features=MAX_FEATURES, 
                                               embedding_size=EMBED_SIZE, 
                                               embedding_file_path=FASTTEXT_EMBEDDINGS_PATH)
    with open(FASTTEXT_INIT_EMBEDDINGS_FILE, 'wb') as f:
        dill.dump(ft_embeddings, f)
else:
    with open(FASTTEXT_INIT_EMBEDDINGS_FILE, 'rb') as f:
        ft_embeddings = dill.load(f)
        
ft_embeddings.shape

(542585, 300)

In [99]:
PARAGRAM_INIT_EMBEDDINGS_FILE = './models/v3-jun19/embeddings/security_model_paragram_init_embeddings.pkl'

if not os.path.isfile(PARAGRAM_INIT_EMBEDDINGS_FILE):
    PARAGRAM_EMBEDDINGS_PATH = './embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    pg_embeddings = load_pretrained_embeddings(word_to_index=word2idx, max_features=MAX_FEATURES, 
                                               embedding_size=EMBED_SIZE, 
                                               embedding_file_path=PARAGRAM_EMBEDDINGS_PATH)
    with open(PARAGRAM_INIT_EMBEDDINGS_FILE, 'wb') as f:
        dill.dump(pg_embeddings, f)
else:
    with open(PARAGRAM_INIT_EMBEDDINGS_FILE, 'rb') as f:
        pg_embeddings = dill.load(f)
        
pg_embeddings.shape 

(542585, 300)

In [100]:
avg_pretrained_embeddings = np.mean([ft_embeddings, pg_embeddings], axis = 0)
avg_pretrained_embeddings.shape

(542585, 300)

In [55]:
from keras.engine.topology import Layer
from keras import backend as K


class AttentionLayer(Layer):
    
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        """
        
        self.supports_masking = True
        self.init = keras.initializers.get('glorot_uniform')

        self.W_regularizer = keras.regularizers.get(W_regularizer)
        self.b_regularizer = keras.regularizers.get(b_regularizer)

        self.W_constraint = keras.constraints.get(W_constraint)
        self.b_constraint = keras.constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(AttentionLayer, self).__init__(**kwargs)
        

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True
        

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    
    def call(self, x, mask=None):
        # TF backend doesn't support it
        # eij = K.dot(x, self.W) 
        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), 
                              K.reshape(self.W, (features_dim, 1))),
                        (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        
        return K.sum(weighted_input, axis=1)

    
    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
    
    
    def get_config(self):
        config = {'step_dim': self.step_dim}
        base_config = super(AttentionLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [56]:
import keras
from keras.utils import multi_gpu_model

def build_gru_model(embedding_matrix, embedding_size, max_len, max_features, gru_units=32):
    
    inp = keras.layers.Input(shape=(max_len,))
    x = keras.layers.Embedding(max_features, embedding_size, 
                                  weights=[embedding_matrix], trainable=True)(inp)
    x = keras.layers.Bidirectional(keras.layers.CuDNNGRU(gru_units*2, return_sequences=True))(x)
    x = keras.layers.Bidirectional(keras.layers.CuDNNGRU(gru_units, return_sequences=True))(x)
    x = AttentionLayer(max_len)(x)
    x = keras.layers.Dense(gru_units*2, activation='relu')(x)
    x = keras.layers.Dropout(rate=0.2)(x)
    x = keras.layers.Dense(gru_units, activation='relu')(x)
    x = keras.layers.Dropout(rate=0.2)(x)

    outp = keras.layers.Dense(1, activation='sigmoid')(x)
    # initialize the model
    model = keras.models.Model(inputs=inp, outputs=outp)

    # make the model parallel
    #model = multi_gpu_model(model, gpus=2)
       
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    
    return model

In [104]:
gru_model = build_gru_model(embedding_matrix=avg_pretrained_embeddings, embedding_size=EMBED_SIZE, 
                            max_len=MAX_LEN, max_features=MAX_FEATURES, gru_units=32)
gru_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         162775500 
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 128)         140544    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1000, 64)          31104     
_________________________________________________________________
attention_layer_1 (Attention (None, 64)                1064      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
__________

In [105]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_y),
                                                 train_y)
class_weights = dict(enumerate(class_weights))
class_weights[1] *= 2
class_weights

{0: 0.5916128396035698, 1: 6.457750269106566}

In [106]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.4,
                              patience=2, min_lr=0.00001)

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, 
                           mode='auto', baseline=None, restore_best_weights=False)

callbacks = [reduce_lr, early_stop]

In [107]:
history = gru_model.fit(train_X, train_y, batch_size=256, epochs=10, callbacks=callbacks,
                        class_weight=class_weights, validation_split=0.1)

/home/redanalyze/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:109: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 162775500 elements. This may consume a large amount of memory.
  num_elements)


Train on 107986 samples, validate on 11999 samples
Epoch 1/10
107986/107986 [==============================] - 397s 4ms/step - loss: 0.4863 - acc: 0.7466 - val_loss: 0.1825 - val_acc: 0.9596
Epoch 2/10
107986/107986 [==============================] - 370s 3ms/step - loss: 0.1581 - acc: 0.9655 - val_loss: 0.1644 - val_acc: 0.9717
Epoch 3/10
107986/107986 [==============================] - 369s 3ms/step - loss: 0.0812 - acc: 0.9815 - val_loss: 0.1777 - val_acc: 0.9579
Epoch 4/10
107986/107986 [==============================] - 369s 3ms/step - loss: 0.0408 - acc: 0.9893 - val_loss: 0.2361 - val_acc: 0.9681
Epoch 5/10
107986/107986 [==============================] - 369s 3ms/step - loss: 0.0197 - acc: 0.9941 - val_loss: 0.2888 - val_acc: 0.9673
Epoch 6/10
107986/107986 [==============================] - 373s 3ms/step - loss: 0.0121 - acc: 0.9963 - val_loss: 0.3896 - val_acc: 0.9694


In [108]:
gru_model.save('./models/v3-jun19/model_files/security_model_train75-jun19.h5')
gru_model.save_weights('./models/v3-jun19/model_files/security_model_train75-jun19_weights.h5')

In [19]:
gru_model = keras.models.load_model('./models/v3-jun19/model_files/security_model_train75-jun19.h5',
                                        custom_objects={'AttentionLayer': AttentionLayer})
pred_y = gru_model.predict([test_X], batch_size=256, verbose=1)

/home/redanalyze/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:109: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 162775500 elements. This may consume a large amount of memory.
  num_elements)


37468/37468 [==============================] - 38s 1ms/step


In [20]:
pred_yr = pred_y.ravel()
pred_yl = [1 if prob > 0.5 else 0 for prob in pred_yr]

In [21]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(y_true=test_y, y_pred=pred_yl)

array([[31308,   369],
       [  353,  5438]])

In [22]:
print(classification_report(y_true=test_y, y_pred=pred_yl))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99     31677
          1       0.94      0.94      0.94      5791

avg / total       0.98      0.98      0.98     37468



# Train on Full Data now!

In [25]:
X_train, X_test, y_train, y_test = train_test_split(norm_docs, labels, 
                                                    test_size=0.0001, random_state=SEED)
len(X_train), len(X_test)

(152135, 16)

In [26]:
import os.path

CVE_WORD2IDX_MAP_FILE = 'models/v3-jun19/embeddings/security_tokenizer_word2idx_fulldata.pkl'

if not os.path.isfile(CVE_WORD2IDX_MAP_FILE):
    tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>', num_words=MAX_FEATURES+1)
    tokenizer.fit_on_texts(list(X_train))
    tokenizer.word_index['<PAD>'] = 0
    with open(CVE_WORD2IDX_MAP_FILE, 'wb') as f:
        dill.dump(tokenizer.word_index, f)
else:
    tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>', num_words=MAX_FEATURES+1)
    with open(CVE_WORD2IDX_MAP_FILE, 'rb') as f:
        word2idx = dill.load(f)
    tokenizer.word_index = word2idx

In [51]:
MAX_FEATURES = len(tokenizer.word_index)
MAX_FEATURES

649383

In [52]:
train_X = tokenizer.texts_to_sequences(X_train)
test_X = tokenizer.texts_to_sequences(X_test)

train_X = keras.preprocessing.sequence.pad_sequences(train_X, maxlen=MAX_LEN)
test_X = keras.preprocessing.sequence.pad_sequences(test_X, maxlen=MAX_LEN)

train_X_lengths = np.array([len(np.nonzero(item)[0]) for item in train_X])
train_X_idx = np.argwhere(train_X_lengths >= 5).ravel()
train_X = train_X[train_X_idx]
train_y = y_train[train_X_idx]

test_X_lengths = np.array([len(np.nonzero(item)[0]) for item in test_X])
test_X_idx = np.argwhere(test_X_lengths >= 5).ravel()
test_X = test_X[test_X_idx]
test_y = y_test[test_X_idx]
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((149939, 1000), (149939,), (16, 1000), (16,))

In [53]:
word2idx = tokenizer.word_index

In [54]:
FASTTEXT_INIT_EMBEDDINGS_FILE = './models/v3-jun19/embeddings/security_model_fasttext_init_embeddings_fulldata.pkl'

if not os.path.isfile(FASTTEXT_INIT_EMBEDDINGS_FILE):
    FASTTEXT_EMBEDDINGS_PATH = './embeddings/fasttext/crawl-300d-2M.vec'
    ft_embeddings = load_pretrained_embeddings(word_to_index=word2idx, max_features=MAX_FEATURES, 
                                               embedding_size=EMBED_SIZE, 
                                               embedding_file_path=FASTTEXT_EMBEDDINGS_PATH)
    with open(FASTTEXT_INIT_EMBEDDINGS_FILE, 'wb') as f:
        dill.dump(ft_embeddings, f)
else:
    with open(FASTTEXT_INIT_EMBEDDINGS_FILE, 'rb') as f:
        ft_embeddings = dill.load(f)
        

PARAGRAM_INIT_EMBEDDINGS_FILE = './models/v3-jun19/embeddings/security_model_paragram_init_embeddings_fulldata.pkl'

if not os.path.isfile(PARAGRAM_INIT_EMBEDDINGS_FILE):
    PARAGRAM_EMBEDDINGS_PATH = './embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    pg_embeddings = load_pretrained_embeddings(word_to_index=word2idx, max_features=MAX_FEATURES, 
                                               embedding_size=EMBED_SIZE, 
                                               embedding_file_path=PARAGRAM_EMBEDDINGS_PATH)
    with open(PARAGRAM_INIT_EMBEDDINGS_FILE, 'wb') as f:
        dill.dump(pg_embeddings, f)
else:
    with open(PARAGRAM_INIT_EMBEDDINGS_FILE, 'rb') as f:
        pg_embeddings = dill.load(f)
        
avg_pretrained_embeddings = np.mean([ft_embeddings, pg_embeddings], axis = 0)

print('FT Embeddings shape:', ft_embeddings.shape)
print('PG Embeddings shape:', pg_embeddings.shape)
print('AVG Embeddings shape:', avg_pretrained_embeddings.shape)

FT Embeddings shape: (649383, 300)
PG Embeddings shape: (649383, 300)
AVG Embeddings shape: (649383, 300)


In [57]:
gru_model = build_gru_model(embedding_matrix=avg_pretrained_embeddings, embedding_size=EMBED_SIZE, 
                            max_len=MAX_LEN, max_features=MAX_FEATURES, gru_units=32)
gru_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         194814900 
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 128)         140544    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1000, 64)          31104     
_________________________________________________________________
attention_layer_1 (Attention (None, 64)                1064      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
__________

In [58]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_y),
                                                 train_y)
class_weights = dict(enumerate(class_weights))
class_weights[1] *= 2
class_weights

{0: 0.5914660123706135, 1: 6.466511407254065}

In [59]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.4,
                              patience=2, min_lr=0.00001)

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=6, 
                           mode='auto', baseline=None, restore_best_weights=False)

callbacks = [reduce_lr, early_stop]

history = gru_model.fit(train_X, train_y, batch_size=256, epochs=10, callbacks=callbacks,
                        class_weight=class_weights, validation_split=0.1)

/home/redanalyze/.local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:109: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 194814900 elements. This may consume a large amount of memory.
  num_elements)


Train on 134945 samples, validate on 14994 samples
Epoch 1/10
134945/134945 [==============================] - 497s 4ms/step - loss: 0.4891 - acc: 0.6904 - val_loss: 0.1849 - val_acc: 0.9540
Epoch 2/10
134945/134945 [==============================] - 490s 4ms/step - loss: 0.1442 - acc: 0.9699 - val_loss: 0.1590 - val_acc: 0.9772
Epoch 3/10
134945/134945 [==============================] - 482s 4ms/step - loss: 0.0696 - acc: 0.9835 - val_loss: 0.2089 - val_acc: 0.9586
Epoch 4/10
134945/134945 [==============================] - 483s 4ms/step - loss: 0.0349 - acc: 0.9904 - val_loss: 0.3053 - val_acc: 0.9707
Epoch 5/10
134945/134945 [==============================] - 490s 4ms/step - loss: 0.0159 - acc: 0.9958 - val_loss: 0.4021 - val_acc: 0.9706
Epoch 6/10
134945/134945 [==============================] - 490s 4ms/step - loss: 0.0104 - acc: 0.9975 - val_loss: 0.4573 - val_acc: 0.9711
Epoch 7/10
134945/134945 [==============================] - 489s 4ms/step - loss: 0.0063 - acc: 0.9985 - val_

In [60]:
gru_model.save('./models/v3-jun19/model_files/security_model_train99-jun19.h5')
gru_model.save_weights('./models/v3-jun19/model_files/security_model_train99-jun19_weights.h5')

In [67]:
pred_y = gru_model.predict([train_X], batch_size=256, verbose=1)

149939/149939 [==============================] - 148s 988us/step


In [ ]:
pred_yr = pred_y.ravel()
pred_yl = [1 if prob > 0.4 else 0 for prob in pred_yr]

In [75]:
confusion_matrix(y_true=train_y, y_pred=pred_yl)

array([[126311,    441],
       [   161,  23026]])

# Deep Learning CPU Inference

In [76]:
def build_gru_cpumodel(embedding_size, max_len, max_features, gru_units=32):
    
    inp = keras.layers.Input(shape=(max_len,))
    x = keras.layers.Embedding(max_features, embedding_size, trainable=True)(inp)
    x = keras.layers.Bidirectional(keras.layers.GRU(gru_units*2, return_sequences=True, 
                                                    reset_after=True, recurrent_activation='sigmoid'))(x)
    x = keras.layers.Bidirectional(keras.layers.GRU(gru_units, return_sequences=True, reset_after=True, 
                                                    recurrent_activation='sigmoid'))(x)
    x = AttentionLayer(max_len)(x)
    x = keras.layers.Dense(gru_units*2, activation='relu')(x)
    x = keras.layers.Dropout(rate=0.2)(x)
    x = keras.layers.Dense(gru_units, activation='relu')(x)
    x = keras.layers.Dropout(rate=0.2)(x)
    outp = keras.layers.Dense(1, activation='sigmoid')(x)

    model = keras.models.Model(inputs=inp, outputs=outp)       
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    
    return model

In [79]:
CVE_WORD2IDX_MAP_FILE = 'models/v3-jun19/embeddings/security_tokenizer_word2idx_fulldata.pkl'
tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>', num_words=MAX_FEATURES+1)
with open(CVE_WORD2IDX_MAP_FILE, 'rb') as f:
    word2idx = dill.load(f)
    tokenizer.word_index = word2idx
MAX_FEATURES = len(tokenizer.word_index)
EMBED_SIZE = 300
MAX_LEN = 1000


gru_cpu_model = build_gru_cpumodel(embedding_size=EMBED_SIZE,
                                   max_len=MAX_LEN, max_features=MAX_FEATURES, gru_units=32)
gru_cpu_model.load_weights('./models/v3-jun19/model_files/security_model_train99-jun19_weights.h5')
gru_cpu_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 1000, 300)         194814900 
_________________________________________________________________
bidirectional_5 (Bidirection (None, 1000, 128)         140544    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 1000, 64)          31104     
_________________________________________________________________
attention_layer_3 (Attention (None, 64)                1064      
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
__________

In [81]:
pred_y = gru_model.predict([train_X], batch_size=256, verbose=1)

149939/149939 [==============================] - 146s 971us/step


In [ ]:
pred_yr = pred_y.ravel()
pred_yl = [1 if prob > 0.4 else 0 for prob in pred_yr]

In [82]:
confusion_matrix(y_true=train_y, y_pred=pred_yl)

array([[126311,    441],
       [   161,  23026]])